# Ways to SIMD

In [122]:
!rm -Rf tmp
!mkdir -p tmp

## Inline Assembly

* [Documentation](https://gcc.gnu.org/onlinedocs/gcc/Extended-Asm.html)
* [Source for "x" register constraint for vector registers](https://stackoverflow.com/a/32667983)

In [5]:
%%writefile tmp/inline-assembly.c

#include <x86intrin.h>
#include <stdio.h>

int main()
{
    float a[8] = {0, 1, 2, 3, 4, 5, 6, 7};
    float b[8] = {7, 6, 5, 4, 3, 2, 1, 0};
    float result[8];

    __m256 avec = _mm256_loadu_ps(a);
    __m256 bvec = _mm256_loadu_ps(b);
    __m256 resultvec;
    
    asm ("vaddps %1, %2, %0"
        : "=x" (resultvec)
        : "x" (avec), "x" (bvec) /* "r" for normal registers, "x"
        : /* clobbers */
        );
    
    _mm256_storeu_ps(result, resultvec);
    
    for (int i = 0; i < 8; ++i)
        printf("%f ", result[i]);
    printf("\n");
}

In [6]:
!cd tmp; gcc -march=sandybridge -c -O inline-assembly.c
!objdump --disassemble tmp/inline-assembly.o

## Intrinsics

In [125]:
%%writefile tmp/intrinsics.c

#include <x86intrin.h>
#include <stdio.h>

int main()
{
    float a[8] = {0, 1, 2, 3, 4, 5, 6, 7};
    float b[8] = {7, 6, 5, 4, 3, 2, 1, 0};
    float result[8];

    __m256 avec = _mm256_loadu_ps(a);
    __m256 bvec = _mm256_loadu_ps(b);
    __m256 resultvec = _mm256_add_ps(avec, bvec);    
    _mm256_storeu_ps(result, resultvec);
    
    for (int i = 0; i < 8; ++i)
        printf("%f ", result[i]);
    printf("\n");
}

In [126]:
!cd tmp; gcc -march=sandybridge -O intrinsics.c -ointrinsics
!./tmp/intrinsics

## Vector types

In [129]:
%%writefile tmp/vector-types.c

#include <stdio.h>

int main()
{
    typedef float v8f __attribute__ ((vector_size (256/8)));
    v8f a = {0, 1, 2, 3, 4, 5, 6, 7};
    v8f b = {7, 6, 5, 4, 3, 2, 1, 0};

    v8f result = a+b+5;
    
    for (int i = 0; i < 8; ++i)
        printf("%f ", result[i]);
    printf("\n");
}

In [130]:
!cd tmp; gcc -march=sandybridge -O vector-types.c -ovector-types
!./tmp/vector-types

In [131]:
%%writefile tmp/vector-types.c

#include <stdio.h>

typedef float v8f __attribute__ ((vector_size (256/8)));

v8f add_vecs(v8f a, v8f b)
{
    return a+b+5;
}

In [132]:
!cd tmp; gcc -march=sandybridge -O -c vector-types.c
! objdump --disassemble tmp/vector-types.o

### One-argument shuffle

In [1]:
%%writefile tmp/vector-types.c

#include <stdio.h>

typedef float v8f __attribute__ ((vector_size (256/8)));
typedef int v8i __attribute__ ((vector_size (256/8)));

v8f reverse_vec(v8f a)
{
    //const v8i idx = {7, 6, 5, 4, 3, 2, 1, 0};
    const v8i idx = {3, 2, 1, 0, 7, 6, 5, 4};
    return __builtin_shuffle(a, idx);
}

In [2]:
!cd tmp; gcc -march=sandybridge -O -c vector-types.c
! objdump --disassemble tmp/vector-types.o

### Two-argument shuffle

In [14]:
%%writefile tmp/vector-types.c

#include <stdio.h>

typedef float v8f __attribute__ ((vector_size (256/8)));
typedef int v8i __attribute__ ((vector_size (256/8)));

v8f interleave(v8f a, v8f b)
{
    // one-argument
    //const v8i idx = {0,1,2,3,4,5,6,7};
    //const v8i idx = {1,2,3,4,5,6,7,0};
    
    // two-argument
    const v8i idx = {2,2+8,3,3+8,6,6+8,7,7+8};
    return __builtin_shuffle(a, b, idx);
}

In [15]:
!cd tmp; gcc -march=sandybridge -O -c vector-types.c
! objdump --disassemble tmp/vector-types.o

## Highway

A SIMD library from Google with a focus on "production" use and broad platform support.

[Design philosophy](https://google.github.io/highway/en/master/design_philosophy.html):

- Portability (e.g. to what extent should AVX-512's ["writemask" vs "zeromask"](https://travisdowns.github.io/blog/2019/12/05/kreg-facts.html) be accommodated?)
    - Encourage "width-agnostic" (`ScalableTag`), i.e. **width not user-specified**
- Performance, but not at all costs (aim within 10-20%, see [instruction matrix](https://github.com/google/highway/blob/1e825be64e11fa52624e524054c3a4bbf4832e77/g3doc/instruction_matrix.pdf))
- Make operation costs visible
- Support multi-versioning with low-cost run-time dispatch

Resources:

- <https://github.com/google/highway>
- [Documentation](https://google.github.io/highway/en/master/)
- [Slides](https://github.com/google/highway/blob/1e825be64e11fa52624e524054c3a4bbf4832e77/g3doc/highway_intro.pdf)

On Debian/Ubuntu: 
```
apt install libhwy-dev
```

In [3]:
%%writefile tmp/highway.cpp

#include <hwy/highway.h>

namespace hn = hwy::HWY_NAMESPACE;

using T = float;

void MulAddLoop(const T* HWY_RESTRICT mul_array,
                const T* HWY_RESTRICT add_array,
                const size_t size, T* HWY_RESTRICT x_array) {
  const hn::ScalableTag<T> d;
  for (size_t i = 0; i < size; i += hn::Lanes(d)) {
    const auto mul = hn::Load(d, mul_array + i);
    const auto add = hn::Load(d, add_array + i);
    auto x = hn::Load(d, x_array + i);
    x = hn::MulAdd(mul, x, add);
    hn::Store(x, d, x_array + i);
  }
}


In [4]:
!cd tmp; g++ -march=native -O -c highway.cpp
!objdump --disassemble tmp/highway.o

## #pragma simd

In [137]:
%%writefile tmp/pragma-simd.c

#include <stdio.h>

typedef float vec_t[8];

void add_them(vec_t a, vec_t b, vec_t result)
{
    #pragma GCC ivdep
    // #pragma omp simd
    // #pragma omp simd aligned(a, b, result:32)
    for (int i = 0; i<8; ++i)
        result[i] = a[i] + b[i];
}

In [138]:
!cd tmp; gcc -march=sandybridge -fopenmp -O -c pragma-simd.c
#!cd tmp; gcc -march=sandybridge -fopenmp -O3 -c pragma-simd.c

!objdump --disassemble tmp/pragma-simd.o

Semantics (openMP 4.5, 2.8.1):

> The `simd` construct enables the execution of multiple iterations of the associated loops
concurrently by means of SIMD instructions.


> A SIMD loop has logical iterations numbered 0,1,...,N-1 where N is the number of loop iterations,
> and the logical numbering denotes the sequence in which the iterations would be executed if the
> associated loop(s) were executed with no SIMD instructions. If the `safelen` clause is used then
> no two iterations executed concurrently with SIMD instructions can have a greater distance in the
> logical iteration space than its value. The parameter of the safelen clause must be a constant
> positive integer expression. If used, the simdlen clause specifies the preferred number of
> iterations to be executed concurrently.

* What does `safelen(12)` mean?

> The parameter of the `simdlen` clause must be a constant
> positive integer. The number of iterations that are executed concurrently at any given time is
> implementation defined. Each concurrent iteration will be executed by a different SIMD lane. Each
> set of concurrent iterations is a SIMD chunk. Lexical forward dependencies in the iterations of the
> original loop must be preserved within each SIMD chunk.
 
* What does `simdlen(16)` mean?
* How do `safelen` and `simdlen` relate to each other

Also:
    
* `#pragma omp declare simd` on *functions*
* with `inbranch`, `notinbranch`

## Scalar Program Instances

In [120]:
%%writefile tmp/scalar-program-instances.ispc

float add_them(float a, float b)
{
    return a + b;
}

In [121]:
!cd tmp; ~/pack/ispc-v1.9.0-linux/ispc  \
    --target=avx2-i32x8 \
    --opt=force-aligned-memory \
    scalar-program-instances.ispc \
    -o scalar-program-instances.o
! objdump --disassemble tmp/scalar-program-instances.o

*  `uniform` and `varying`
*  `programCount` and `programIndex`
*  `x16` targets... why?
*  How do you think shuffles are done?
*  Compare this with `omp declare simd`, including `uniform`.